In [12]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import sklearn
from scipy import ndimage, misc
import math
%matplotlib inline
import cv2
import time
import glob

img_list = []
filenamelist= []
for img in glob.glob("0525_images\\CCD-4\\*.jpg"):
    #img_list.append(cv2.imread(img,0))
    filenamelist.append(img)
np.array(filenamelist).shape

(152,)

In [ ]:
for iter_file in range(len(filenamelist)):
    start = time.time()
    filename = filenamelist[iter_file]
    print(filename)
    image = np.array(cv2.imread(filename,0))
    backtorgb = cv2.cvtColor(image,cv2.COLOR_GRAY2RGB)


    #=======================以下找出真正的O-ring=============================
    ret, thresh1 = cv2.threshold(image, 200, 255, cv2.THRESH_BINARY)
    #find contours1
    _ , contours, hierarchy = cv2.findContours(thresh1,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    contours_final = []
    approx_list = []
    for i in range(np.array(contours).shape[0]):
            if(cv2.contourArea(contours[i])>300000 and cv2.contourArea(contours[i])<800000):
                contours_final.append(contours[i])
                #cv2.drawContours(backtorgb,contours[i],-1,(0,0,255),3)

                #print("Area: ",cv2.contourArea(contours[i]))
                epsilon = 0.5# * cv2.arcLength(contours[i],True)
                approx = cv2.approxPolyDP(contours[i], epsilon, True)
                approx_list.append(approx)
                #cv2.polylines(backtorgb, [approx], True, (255, 0, 0), 2)
    #====================外圈減內圈得到真的O-ring================================
    inner_contour_img = np.zeros_like(image)
    cv2.drawContours(inner_contour_img, [approx_list[1]],-1,(255,255,255),-1)
    inner_contour_img = 1 - inner_contour_img

    outer_contour_img = np.zeros_like(image)
    cv2.drawContours(outer_contour_img, [approx_list[0]],-1,(255,255,255),-1)
    outer_contour_img = 1 - outer_contour_img

    # for below delete contour after algorithm
    delete_contour_img = np.zeros_like(image)
    cv2.drawContours(delete_contour_img, [approx_list[0]],-1,(255,255,255),3)

    #outer - inner
    oringmask = outer_contour_img- inner_contour_img

    mask_image = oringmask*image
    #mask_image = 1 - mask_image
    image[oringmask != 1] =255
    
    #=================================用canny找出輪廓==================
    image = cv2.blur(image,(3,3))
    kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
    image = cv2.filter2D(image, -1, kernel)

    image = cv2.GaussianBlur(image,(7,7),0,0)
    
    image = cv2.Canny(image,60,180,3)
    #==================用findcontours把canny得到的 ROI切出來===============

    _ , contours, hierarchy = cv2.findContours(image,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    inner_contours_final = []
    approx_list = []
    index = []
    for i in range(np.array(contours).shape[0]):
        if (hierarchy[0][i][3] >= 0):
            if(cv2.contourArea(contours[i])<500):
                #cv2.drawContours(backtorgb, contours, i, (0, 255, 0), 1, 8);
                #inner_contours_final.append(contours[i])
                epsilon = 0.0000# * cv2.arcLength(contours[i],True)
                index.append(i)
                approx = cv2.approxPolyDP(contours[i], epsilon, True)
                #print(cv2.contourArea(approx))
                approx_list.append(approx)
                #print(np.array(approx_list).shape)
                cv2.polylines(backtorgb, [approx], True, (0,0,255), 5)
    #=================================================================
    end = time.time()
    print(end - start)
    print(".\\Detect\\Stop4\\" +filenamelist[iter_file][18:-4]+'_Detect.jpg')
    cv2.imwrite(".\\Detect\\Stop4\\" +filenamelist[iter_file][18:-4]+'_Detect.jpg',backtorgb)


0525_images\CCD-4\1.jpg
0.14684629440307617
.\Detect\Stop4\1_Detect.jpg
0525_images\CCD-4\10.jpg
0.14312529563903809
.\Detect\Stop4\10_Detect.jpg
0525_images\CCD-4\100.jpg
0.1466081142425537
.\Detect\Stop4\100_Detect.jpg
0525_images\CCD-4\101.jpg
0.147627592086792
.\Detect\Stop4\101_Detect.jpg
0525_images\CCD-4\102.jpg
0.15056657791137695
.\Detect\Stop4\102_Detect.jpg
0525_images\CCD-4\103.jpg
0.1685621738433838
.\Detect\Stop4\103_Detect.jpg
0525_images\CCD-4\104.jpg
0.15062618255615234
.\Detect\Stop4\104_Detect.jpg
0525_images\CCD-4\105.jpg
0.1376798152923584
.\Detect\Stop4\105_Detect.jpg
0525_images\CCD-4\106.jpg
0.14560389518737793
.\Detect\Stop4\106_Detect.jpg
0525_images\CCD-4\107.jpg
0.14613699913024902
.\Detect\Stop4\107_Detect.jpg
0525_images\CCD-4\108.jpg
0.16755342483520508
.\Detect\Stop4\108_Detect.jpg
0525_images\CCD-4\109.jpg
0.15954160690307617
.\Detect\Stop4\109_Detect.jpg
0525_images\CCD-4\11.jpg
0.14358973503112793
.\Detect\Stop4\11_Detect.jpg
0525_images\CCD-4\110.jpg